## 引子

&emsp;&emsp;在`K`近邻学习中，需要任意样本$x$附近任意小距离之内总能找到一个训练样本，及训练样本的采样密度足够大。想要满足上述关系的话，取最小距离为$\frac{1}{1000}$, 那么在`1`维空间中，需要$1000$个样本，`2`维空间内，需要$1000^{2}$个样本...。

## 降维方法

### **主成分分析**(PCA)

&emsp;&emsp;线性降维是寻找一个合适的基向量矩阵$P$, 使得$Y=PX$, 其中$Y$是样本在新空间的表达，$P$是基向量，$X$是原始样本。假设我们有一个数据$x_{1}$, 它具有$m$个特征$x_{1} = [x_{11}, x_{12}, \cdots, x_{1m}]$，我们还有一个正交矩阵$[\alpha_{1}, \alpha_{2}, \cdots, \alpha_{k}]$, (里面每一列都是一个正交基$\alpha_{1}=[\alpha_{11}, \alpha_{21}, \cdots, \alpha_{m1}]$)，它的维度为$m \times k$，那么它与$x$乘完之后，就是一个$1 \times k$的向量，当$k$小于$m$时，就达到了数据降维的目的。

&emsp;&emsp;假设我们有$R$个基：$\left(\begin{array}{c}p_{1} \\ p_{2} \\ \vdots \\ p_{R}\end{array}\right)$，$p_{i} \in \mathbb{R}^{1 * N}$是一个行向量，表示第$i$个基，$a_{j} \in \mathbb{R}^{N * 1}$是一个列向量，表示第$j$个原始数据。矩阵表示为：

$$
\left(\begin{array}{c}
p_{1} \\
p_{2} \\
\vdots \\
p_{R}
\end{array}\right)\left(\begin{array}{llll}
a_{1} & a_{2} & \cdots & a_{M}
\end{array}\right)=\left(\begin{array}{cccc}
p_{1} a_{1} & p_{1} a_{2} & \cdots & p_{1} a_{M} \\
p_{2} a_{1} & p_{2} a_{2} & \cdots & p_{2} a_{M} \\
\vdots & \vdots & \ddots & \vdots \\
p_{R} a_{1} & p_{R} a_{2} & \cdots & p_{R} a_{M}
\end{array}\right)
$$

&emsp;&emsp;这样就可以将一个$N$维的数据变换到更低维度的空间$R$中去，变换后的维度取决于基的数量。从原样本$X \in \mathbb{R}^{N * M}$降维到$Y \in \mathbb{R}^{R * M}$。

&emsp;&emsp;**直观理解就是将右边$X$矩阵中的每一列列向量变换到左边矩阵中每一行行向量为基所表示的空间中去**。


&emsp;&emsp;**选择基向量的衡量标准是什么**？如何选择基向量是最优的？需要满足两点：1. **最近重构性**：样本点到这条直线的距离足够近。2. **最大可分性**：样本点在这个直线的投影尽量分散。

<img src="../images/pca.png" width="20%">

&emsp;&emsp;样本点投影后到原点的距离为$a$。样本点到直线的投影距离为$b$。依据勾股定理我们有$c^{2} = a^{2} + b^{2}$。若有$m$个样本点的话，我们有$\sum_{i=1}^{m}c^{2} =  \sum_{i=1}^{m} a^{2} + \sum_{i=1}^{m}b^{2}$。

&emsp;&emsp;**最大化数据映射后的方差**：`PCA`的两个条件中，想要样本点到这条直线的距离足够近的话，就是期望$\sum_{i=1}^{m}b^{2}$足够小。样本点在这个直线的投影尽量分散的话，就是期望投影之后数据的方差足够大。如果处理数据之前已经将数据做了归一化，则$\sum_{i=1}^{m} a^{2}$可以看作是每个样本点的方差的求和，又由于$c$是定值，所以上述这两个条件是等价的。我们期望方差$\frac{1}{m}\sum_{i=1}^{m}a^{2}$足够大。这个**最大化基变量后的期望方差就是我们需要去优化的目标**。

- **举例**：假设我们有`5`条数据，每条数据的特征维度为`2`。用矩阵表示为：

$$
\left(\begin{array}{lllll}
1 & 1 & 2 & 4 & 2 \\
1 & 3 & 3 & 4 & 4
\end{array}\right)
$$

In [1]:
import numpy as np
X_origin = np.array([[1, 1], [1, 3], [2, 3], [4, 4], [2, 4]])
X_origin

array([[1, 1],
       [1, 3],
       [2, 3],
       [4, 4],
       [2, 4]])

&emsp;&emsp;依据方差公式$\operatorname{Var}(a)=\frac{1}{m} \sum_{i=1}^{m}\left(a_{i}-\mu\right)^{2}$计算其方差，在计算方差之前需要先需要计算其减去均值的值：

$$
\left(\begin{array}{lllll}
-1 & -1 & 0 & 2 & 0 \\
-2 & 1 & 1 & 2 & 2
\end{array}\right)
$$

&emsp;&emsp;若算完了均值的话，方差计算公式则可以表示为：$\operatorname{Var}(a)=\frac{1}{m} \sum_{i=1}^{m}\left(a_{i}\right)^{2}$。

In [2]:
X_mean = X_origin - np.mean(X_origin, axis=0)
X_mean

array([[-1., -2.],
       [-1.,  0.],
       [ 0.,  0.],
       [ 2.,  1.],
       [ 0.,  1.]])

&emsp;&emsp;对于上面二维降成一维的问题来说，找到那个使得方差最大的方向就可以了。不过对于更高维，还有一个问题需要解决。考虑三维降到二维问题。与之前相同，首先我们希望找到一个方向使得投影后方差最大，这样就完成了第一个方向的选择，继而我们选择第二个投影方向。

&emsp;&emsp;如果我们还是单纯只选择方差最大的方向，很明显，这个方向与第一个方向应该是“几乎重合在一起”，显然这样的维度是没有用的，因此，应该有其他约束条件。从直观上说，让两个特征尽可能表示更多的原始信息，我们是**不希望投影之后它们之间存在（线性）相关性的**，因为相关性意味着两个字段不是完全独立，必然存在重复表示的信息。

&emsp;&emsp;协方差公式为：

$$
\operatorname{Cov}(a, b)=\frac{1}{m-1} \sum_{i=1}^{m}\left(a_{i}-\mu_{a}\right)\left(b_{i}-\mu_{b}\right)
$$

&emsp;&emsp;由于已经让每个字段均值为0，则：

$$
\operatorname{Cov}(a, b)=\frac{1}{m} \sum_{i=1}^{m}a_{i}b_{i}
$$

&emsp;&emsp;当样本数较大时，不必在意其是 m 还是 m-1，为了方便计算，我们分母取 m。当协方差为 0 时，表示两个变量完全独立。为了让协方差为 0，我们选择第二个基时只能在与第一个基正交的方向上进行选择，因此最终选择的两个方向一定是正交的。

&emsp;&emsp;至此，我们得到了降维问题的优化目标：**将一组N维向量降为K维（K大于0，小于N），其目标是选择K个单位（模为1）正交基，使得原始数据变换到这组基上后，各特征两两间协方差为0，而特征的方差则尽可能大（在正交的约束下，取最大的K个方差）**。



- **协方差矩阵**

&emsp;&emsp;上面我们导出了优化目标，但是这个目标似乎不能直接作为操作指南（或者说算法），因为它只说要什么，但根本没有说怎么做。所以我们要继续在数学上研究计算方案。我们看到，最终要达到的目的与字段内方差及字段间协方差有密切关系。因此我们希望能将两者统一表示，仔细观察发现，两者均可以表示为内积的形式，而内积又与矩阵相乘密切相关。于是我们来了灵感：

&emsp;&emsp;假设我们只有$a$和$b$两个字段，那么我们将它们按行组成矩阵$X$：

$$
X=\left(\begin{array}{llll}
a_{1} & a_{2} & \cdots & a_{m} \\
b_{1} & b_{2} & \cdots & b_{m}
\end{array}\right)
$$

&emsp;&emsp;然后我们用$X$乘以$X$的转置，并乘上系数$\frac{1}{m}$：

$$
\frac{1}{m} X X^{\top}=\left(\begin{array}{cc}
\frac{1}{m} \sum_{i=1}^{m} a_{i}^{2} & \frac{1}{m} \sum_{i=1}^{m} a_{i} b_{i} \\
\frac{1}{m} \sum_{i=1}^{m} a_{i} b_{i} & \frac{1}{m} \sum_{i=1}^{m} b_{i}^{2}
\end{array}\right)
$$

&emsp;&emsp;设我们有$m$个$n$维数据记录，将其按列排成$n$乘$m$的矩阵$X$，设$C=\frac{1}{m}XX^{𝖳}$，则$C$是一个对称矩阵，其对角线分别个各个字段的方差，而第$i$行$j$列和$j$行$i$列元素相同，表示$i$和$j$两个字段的协方差。

&emsp;&emsp;根据上述推导，我们发现要达到优化目前，等价于**将协方差矩阵对角化：即除对角线外的其它元素化为0，并且在对角线上将元素按大小从上到下排列**，这样我们就达到了优化目的。这样说可能还不是很明晰，我们进一步看下原矩阵与基变换后矩阵协方差矩阵的关系：

&emsp;&emsp;设原始数据矩阵$X$对应的协方差矩阵为$C$，而$P$是一组基按行组成的矩阵，设$Y=PX$，则$Y$为$X$对$P$做基变换后的数据。设$Y$的协方差矩阵为$D$，我们推导一下$D$与$C$的关系。

$$
\begin{aligned}
D &=\frac{1}{m} Y Y^{\top} \\
&=\frac{1}{m}(P X)(P X)^{\top} \\
&=\frac{1}{m} P X X^{\top} P^{\top} \\
&=P\left(\frac{1}{m} X X^{\top}\right) P^{\top} \\
&=P C P^{\top}
\end{aligned}
$$

&emsp;&emsp;现在事情很明白了！我们要找的$P$不是别的，而是能让原始协方差矩阵对角化的$P$。换句话说，优化目标变成了**寻找一个矩阵$P$，满足$PCP^{𝖳}$是一个对角矩阵，并且对角元素按从大到小依次排列，那么$P$的前$K$行就是要寻找的基，用$P$的前$K$行组成的矩阵乘以$X$就使得$X$从$N$维降到了$K$维并满足上述优化条件**。

- **转换成拉格朗日乘子法求解**

&emsp;&emsp;我们知道样本点$x_{i}$在基$w$下的坐标为$(x_{i}, w) = x_{i}^{T}x$, 于是我们有方差：

$$
\begin{aligned}
D(x) &=\frac{1}{m} \sum_{i=1}^{m}\left(x_{i}^{T} w\right)^{2} \\
&=\frac{1}{m} \sum_{i=1}^{m}\left(x_{i}^{T} w\right)^{T}\left(x_{i}^{T} w\right) \\
&=\frac{1}{m} \sum_{i=1}^{m} w^{T} x_{i} x_{i}^{T} w \\
&=w^{T}\left(\frac{1}{m} \sum_{i=1}^{m} x_{i} x_{i}^{T}\right) w
\end{aligned}
$$

&emsp;&emsp;我们看到$\left(\frac{1}{m} \sum_{i=1}^{m} x_{i} x_{i}^{T}\right)$就是原样本的协方差，令其为$\Lambda$, 于是我们有：

$$
\left\{\begin{array}{l}
\max \left\{w^{T} \Lambda w\right\} \\
s . t . w^{T} w=1
\end{array}\right.
$$

&emsp;&emsp;然后构造拉格朗日函数：

$$
L(w)=w^{T} \Lambda w+\lambda\left(1-w^{T} w\right)
$$

&emsp;&emsp;对$w$求导，并令值其为0：

$$
\lambda w = \lambda w
$$

&emsp;&emsp;此时方差为：

$$
D(x)=w^{T} \Lambda w=\lambda w^{T} w=\lambda
$$

&emsp;&emsp;于是我们发现，$x$投影后的方差就是协方差矩阵的特征值。我们要找到最大方差也就是协方差矩阵最大的特征值，最佳投影方向就是最大特征值所对应的特征向量，次佳就是第二大特征值对应的特征向量，以此类推。

&emsp;&emsp;算法流程：

<img src="../images/pca_algorithms.png" width="50%">

### 实例

&emsp;&emsp;以下述数据为例，用`PCA`方法将这组二维数据降到一维。

$$
\left(\begin{array}{lllll}
-1 & -1 & 0 & 2 & 0 \\
-2 & 1 & 1 & 2 & 2
\end{array}\right)
$$

&emsp;&emsp;因为这个矩阵的每行已经是零均值，这里我们直接求协方差矩阵：

$$
C=\frac{1}{5}\left(\begin{array}{ccccc}
-1 & -1 & 0 & 2 & 0 \\
-2 & 0 & 0 & 1 & 1
\end{array}\right)\left(\begin{array}{cc}
-1 & -2 \\
-1 & 0 \\
0 & 0 \\
2 & 1 \\
0 & 1
\end{array}\right)=\left(\begin{array}{cc}
\frac{6}{5} & \frac{4}{5} \\
\frac{4}{5} & \frac{6}{5}
\end{array}\right)
$$

&emsp;&emsp;然后求其特征值和特征向量:

$$
\lambda_{1}=2, \lambda_{2}=2 / 5
$$

&emsp;&emsp;其对应的特征向量分别是：

$$
c_{1}\left(\begin{array}{l}
1 \\
1
\end{array}\right), c_{2}\left(\begin{array}{c}
-1 \\
1
\end{array}\right)
$$

&emsp;&emsp;其中对应的特征向量分别是一个通解，$c_{1}$和$c_{2}$可取任意实数。那么标准化后的特征向量为：

$$
\left(\begin{array}{c}
1 / \sqrt{2} \\
1 / \sqrt{2}
\end{array}\right),\left(\begin{array}{c}
-1 / \sqrt{2} \\
1 / \sqrt{2}
\end{array}\right)
$$

&emsp;&emsp;因此我们的矩阵$P$是：

$$
P=\left(\begin{array}{cc}
1 / \sqrt{2} & 1 / \sqrt{2} \\
-1 / \sqrt{2} & 1 / \sqrt{2}
\end{array}\right)
$$

&emsp;&emsp;可以验证协方差矩阵$C$的对角化：

$$
P C P^{\top}=\left(\begin{array}{cc}
1 / \sqrt{2} & 1 / \sqrt{2} \\
-1 / \sqrt{2} & 1 / \sqrt{2}
\end{array}\right)\left(\begin{array}{cc}
6 / 5 & 4 / 5 \\
4 / 5 & 6 / 5
\end{array}\right)\left(\begin{array}{cc}
1 / \sqrt{2} & -1 / \sqrt{2} \\
1 / \sqrt{2} & 1 / \sqrt{2}
\end{array}\right)=\left(\begin{array}{cc}
2 & 0 \\
0 & 2 / 5
\end{array}\right)
$$

&emsp;&emsp;最后我们用P的第一行乘以数据矩阵，就得到了降维后的表示：

$$
Y=\left(\begin{array}{ll}
1 / \sqrt{2} & 1 / \sqrt{2}
\end{array}\right)\left(\begin{array}{ccccc}
-1 & -1 & 0 & 2 & 0 \\
-2 & 0 & 0 & 1 & 1
\end{array}\right)=\left(\begin{array}{lllll}
-3 / \sqrt{2} & -1 / \sqrt{2} & 0 & 3 / \sqrt{2} & -1 / \sqrt{2}
\end{array}\right)
$$

- **代码实现**：

In [3]:
class PCA(object):
    def __init__(self):
        super(PCA, self).__init__()

    def calculate_covariance_matrix(self, X, Y=None):
        # 计算协方差矩阵
        m = X.shape[0]
        X = X - np.mean(X, axis=0)
        Y = X if Y == None else Y - np.mean(Y, axis=0)
        return 1 / m * np.matmul(X.T, Y)

    def transform(self, X, n_components):
        X = X - np.mean(X, axis=0)
        # 设n=X.shape[1]，将n维数据降维成n_component维
        covariance_matrix = self.calculate_covariance_matrix(X)
        print("协方差矩阵为: \n", covariance_matrix)
        # 获取特征值，和特征向量
        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
        print("特征值: \n", eigenvalues)
        print("特征向量: \n", eigenvectors)
        # 对特征向量排序，并取最大的前n_component组
        # print(np.matmul(np.matmul(eigenvectors, covariance_matrix), eigenvectors.T))
        idx = eigenvalues.argsort()[::-1]
        eigenvectors = eigenvectors[:, idx]
        
        eigenvectors = eigenvectors[:, :n_components]
        # 转换
        return np.matmul(X, eigenvectors)

In [4]:
print("X", X_mean)
print("降维后的数据：\n", PCA().transform(X_origin, 1))

X [[-1. -2.]
 [-1.  0.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  1.]]
协方差矩阵为: 
 [[1.2 0.8]
 [0.8 1.2]]
特征值: 
 [2.  0.4]
特征向量: 
 [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
降维后的数据：
 [[-2.12132034]
 [-0.70710678]
 [ 0.        ]
 [ 2.12132034]
 [ 0.70710678]]


&emsp;&emsp;总结来说就是：`PCA`的工作就是从原始的空间中顺序地找一组相互正交的坐标轴，新的坐标轴的选择与数据本身是密切相关的。其中，第一个新坐标轴选择是原始数据中方差最大的方向，第二个新坐标轴选取是与第一个坐标轴正交的平面中使得方差最大的，第三个轴是与第`1`,`2`个轴正交的平面中方差最大的。依次类推，可以得到$n$个这样的坐标轴。通过这种方式获得的新的坐标轴，我们发现，大部分方差都包含在前面$k$个坐标轴中，后面的坐标轴所含的方差几乎为`0`。于是，我们可以忽略余下的坐标轴，只保留前面k个含有绝大部分方差的坐标轴。事实上，这相当于只保留包含绝大部分方差的维度特征，而忽略包含方差几乎为`0`的特征维度，实现对数据特征的降维处理。

- 参考

1. http://blog.codinglabs.org/articles/pca-tutorial.html
2. https://zhuanlan.zhihu.com/p/32183010
3. https://zhuanlan.zhihu.com/p/32412043
4. https://zhuanlan.zhihu.com/p/77151308


2. **多维缩放法**(MDS)

### 奇异值分解

### 因子分析(FA)

### 独立成分分析(ICA)

### 非线性降维


1. 核化线性降维
2. 流形学

## 度量学习

度量学习是将降维作为一种学习的方法。